In [1]:
%load_ext pycodestyle_magic

In [2]:
# %%pycodestyle

# Required libraries
import datetime as dt
import matplotlib.pyplot as plt

import os
import math
import time
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(font_scale=1.5)

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

start = time.time()

In [3]:
# Constants
my_data = "../data"

# Load data

In [4]:
# Load data of known customers
datafile = "datazon_customer_data.csv"
full_path = os.path.join(my_data, datafile)
df_customers = pd.read_csv(full_path, dtype={'CustomerID': str,
                                         'InvoiceNumberYear': 'int64',
                                         'LastInvoiceMonth': str,
                                         'LastInvoiceMonthNb': int,
                                         'MeanLinesPerInvoice': float,
                                         'MeanQuantityPerInvoice': float,
                                         'MeanAmountPerInvoice': float,
                                         'TotalAmountSpent': float,
                                         'cluster_k': 'int64'})
print(df_customers.shape)

(4323, 9)


In [5]:
# Load data of new customer orders
datafile = "datazon_random_invoices.csv"
full_path = os.path.join(my_data, datafile)
df_sales = pd.read_csv(full_path, dtype={'CustomerID': str})
df_sales['InvoiceDate'] = df_sales['InvoiceDate'].astype('datetime64')
print(df_sales.shape)


(2267, 8)


### InvoiceDate
This feature is useful to calculate the frequency of purchase of a customer, and since how long he didn't make any purchase.

In [6]:
print('Earliest invoice date: {0}'.format(df_sales['InvoiceDate'].min()))
print('Latest invoice date: {0}'.format(df_sales['InvoiceDate'].max()))

Earliest invoice date: 2018-01-02 00:00:00
Latest invoice date: 2018-12-24 00:00:00


# Create features to qualify customers

### Number of orders per year

In [7]:
# Calculate number of invoices per year. Of course, this cannot be used for
# new customers but is still useful to classify existing ones.
invoice_number = (df_sales[['CustomerID', 'InvoiceNo']].drop_duplicates()
                  .groupby(by=['CustomerID']).count().reset_index())
invoice_number.rename(index=str, columns={'InvoiceNo': 'InvoiceNumberYear'},
                      inplace=True)
invoice_number.shape

(29, 2)

### Last invoice date

In [8]:
# Calculate last invoice date
last_invoice_date = (df_sales[['CustomerID', 'InvoiceDate']]
                     .groupby(by=['CustomerID']).max().reset_index())
# Replace by year/month
last_invoice_date['InvoiceDate'] = (last_invoice_date['InvoiceDate'].dt.year*100
                                    + last_invoice_date['InvoiceDate'].dt.month)

last_invoice_date.rename(index=str, columns={'InvoiceDate': 'LastInvoiceMonth'},
                         inplace=True)

last_invoice_date.shape

(29, 2)

In [9]:
# Calculate number of months between last invoice and "today"
# "today" is defined as the last month of the dataset.
max_date = last_invoice_date['LastInvoiceMonth'].max()
last_invoice_date['LastInvoiceMonthNb'] = max_date - last_invoice_date['LastInvoiceMonth']

### Number of line items per invoice per customer

In [10]:
# Count number of lines on each invoice
invoice_lines = (df_sales[['CustomerID', 'InvoiceNo', 'StockCode']]
                 .groupby(by=['CustomerID', 'InvoiceNo']).count().reset_index())

# Calculate mean value per customer of number of lines on ivoice
invoice_lines_mean = (invoice_lines[['CustomerID', 'StockCode']]
                      .groupby(by=['CustomerID']).mean().reset_index())
invoice_lines_mean.rename(index=str, columns={'StockCode': 'MeanLinesPerInvoice'},
                          inplace=True)
invoice_lines_mean.shape

(29, 2)

### Sum of quantities per order

In [11]:
# Calculate total number of quantities for each invoice
invoice_items = (df_sales[['CustomerID', 'InvoiceNo', 'Quantity']]
                 .groupby(by=['CustomerID', 'InvoiceNo']).sum().reset_index())

# Calculate mean value per customer of number of quantity of items per ivoice
invoice_items_mean = (invoice_items[['CustomerID', 'Quantity']]
                      .groupby(by=['CustomerID']).mean().reset_index())
invoice_items_mean.rename(index=str, columns={'Quantity': 'MeanQuantityPerInvoice'},
                          inplace=True)


### Mean amount of invoice, per customer

In [12]:
# Calculate total invoice amount for each invoice
df_sales['TotalPrice'] = df_sales['Quantity'] * df_sales['UnitPrice']
invoice_amount = (df_sales[['CustomerID', 'InvoiceNo', 'TotalPrice']]
                  .groupby(by=['CustomerID', 'InvoiceNo']).sum().reset_index())

# Calculate mean monetary value for invoice, per customer
invoice_amount_mean = (invoice_amount[['CustomerID', 'TotalPrice']]
                       .groupby(by=['CustomerID']).mean().reset_index())
invoice_amount_mean.rename(index=str, columns={'TotalPrice': 'MeanAmountPerInvoice'},
                           inplace=True)


### Total amount spent

In [13]:
# Calculate mean monetary value for invoice, per customer
total_amount_spent = (invoice_amount[['CustomerID', 'TotalPrice']]
                      .groupby(by=['CustomerID']).sum().reset_index())
total_amount_spent.rename(index=str, columns={'TotalPrice': 'TotalAmountSpent'},
                          inplace=True)


### Create customer dataframe

In [14]:
df_new_customers = invoice_number.join(last_invoice_date.set_index('CustomerID'),
                                       on='CustomerID')
df_new_customers = df_new_customers.join(invoice_lines_mean.set_index('CustomerID'),
                                         on='CustomerID')
df_new_customers = df_new_customers.join(invoice_items_mean.set_index('CustomerID'),
                                         on='CustomerID')
df_new_customers = df_new_customers.join(invoice_amount_mean.set_index('CustomerID'),
                                         on='CustomerID')
df_new_customers = df_new_customers.join(total_amount_spent.set_index('CustomerID'),
                                         on='CustomerID')

## Create customer segmentations

This is based on the total set of known customers.

In [15]:
# Separate features and labels for training data
X = df_customers[['InvoiceNumberYear', 'LastInvoiceMonthNb',
                  'MeanLinesPerInvoice', 'MeanAmountPerInvoice',
                  'TotalAmountSpent']]
y = df_customers['cluster_k'].astype('int64')

X_pred = df_new_customers[['InvoiceNumberYear', 'LastInvoiceMonthNb',
                           'MeanLinesPerInvoice', 'MeanAmountPerInvoice',
                           'TotalAmountSpent']]


In [16]:
# Random forest classifier
rfc = RandomForestClassifier(n_estimators=200)

model = rfc.fit(X, y)

y_pred = rfc.predict(X_pred)

df_new_customers['cluster_k'] = y_pred

In [17]:
df_new_customers.sort_values('cluster_k')

,CustomerID,InvoiceNumberYear,LastInvoiceMonth,LastInvoiceMonthNb,MeanLinesPerInvoice,MeanQuantityPerInvoice,MeanAmountPerInvoice,TotalAmountSpent,cluster_k
14,customer22,3,201809,3,41.666667,498.000000,1646.870000,4940.61,0
26,customer7,5,201811,1,33.600000,257.000000,734.920000,3674.60,0
25,customer6,2,201811,1,38.500000,400.000000,1511.980000,3023.96,0
4,customer13,2,201811,1,38.000000,223.000000,940.545000,1881.09,0
23,customer4,2,201812,0,30.000000,274.000000,464.160000,928.32,0
22,customer3,7,201811,1,28.142857,212.428571,676.544286,4735.81,0
9,customer18,3,201809,3,37.333333,283.666667,1045.146667,3135.44,0
18,customer26,1,201807,5,35.000000,542.000000,1370.280000,1370.28,0
24,customer5,2,201806,6,12.000000,379.000000,2257.360000,4514.72,1
21,customer29,1,201802,10,23.000000,687.000000,1951.450000,1951.45,1


In [18]:
df_new_customers.groupby('cluster_k').mean()

,InvoiceNumberYear,LastInvoiceMonth,LastInvoiceMonthNb,MeanLinesPerInvoice,MeanQuantityPerInvoice,MeanAmountPerInvoice,TotalAmountSpent
cluster_k,,,,,,,
0,3.125,201810.125,1.875,35.280357,336.261905,1048.805744,2961.26375
1,1.250,201803.250,8.750,23.250000,357.500000,1489.832500,2054.17250
4,4.125,201809.625,2.375,15.573810,163.127083,421.054597,1698.84000
7,4.000,201806.000,6.000,64.500000,687.250000,2203.515000,8814.06000
